# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
import pickle
import re
import nltk
from sqlalchemy import create_engine
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, f1_score, fbeta_score, classification_report
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

!pip install xgboost
import xgboost as xgb

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('df', con=engine)

df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [4]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text, remove_url=True, remove_punctuation=True, remove_stopwords=False):
    if remove_url == True:
        # Detect all the urls from the text 
        detected_urls = re.findall(url_regex, text)

        # Replace all urls with a placeholder string
        for url in detected_urls:
            text = text.replace(url, "urlplaceholder")
            
    if remove_punctuation == True:
        # remove punctuation
        text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize, normalize case, remove stop words and whitespaces
    if remove_stopwords == True:
        tokens = [lemmatizer.lemmatize(word).lower().strip() for word in tokens if word not in stop_words]
    else:
        tokens = [lemmatizer.lemmatize(word).lower().strip() for word in tokens]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('trunc', TruncatedSVD(n_components=100)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fe3a5d44620>, vocabulary=None)),
  ('trunc', TruncatedSVD(algorithm='randomized', n_components=100, n_iter=5,
          random_state=None, tol=0.0)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
     

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
# split data in train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# train classifier
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

def class_report(y_test, y_pred):
    for i, column in enumerate(y_test):
        print(column)
        print(classification_report(y_test[column], y_pred[:, i]))
        print('#####################################################\n')
        
class_report(y_test, y_pred)

related
             precision    recall  f1-score   support

          0       0.50      0.30      0.37      1266
          1       0.80      0.90      0.85      3978

avg / total       0.73      0.76      0.74      5244

#####################################################

request
             precision    recall  f1-score   support

          0       0.87      0.98      0.92      4309
          1       0.76      0.30      0.43       935

avg / total       0.85      0.86      0.83      5244

#####################################################

offer
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      5178
          1       0.64      0.11      0.18        66

avg / total       0.98      0.99      0.98      5244

#####################################################

aid_related
             precision    recall  f1-score   support

          0       0.67      0.84      0.75      3073
          1       0.65      0.43      0.52      2

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# class StartingVerbExtractor(BaseEstimator, TransformerMixin):

#     def starting_verb(self, text):
#         sentence_list = nltk.sent_tokenize(text)
#         for sentence in sentence_list:
#             pos_tags = nltk.pos_tag(tokenize(sentence))
#             first_word, first_tag = pos_tags[0]
#             if first_tag in ['VB', 'VBP'] or first_word == 'RT':
#                 return True
#         return False

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         X_tagged = pd.Series(X).apply(self.starting_verb)
#         return pd.DataFrame(X_tagged)
    

# pipeline_cv = Pipeline([
#     ('features', FeatureUnion([
#         ('text_pipeline', Pipeline([
#             ('vect', CountVectorizer(tokenizer=tokenize)),
#             ('tfidf', TfidfTransformer())
#         ])),
        
#         ('starting_verb', StartingVerbExtractor())
#     ])),
    
#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
# ])

# pipeline_cv.get_params()

In [9]:
pipeline_cv = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

pipeline_cv.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7ff73a069048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [10]:
%%time

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.75, 1.0),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [10, 50],
    'clf__estimator__min_samples_split': [2, 4],
}

cv = GridSearchCV(pipeline_cv, param_grid=parameters, verbose=2)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1), total=  53.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1), total=  55.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1), total=  54.0s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 2), total= 2.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__max_df

[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed: 417.5min finished


CPU times: user 7h 3min 13s, sys: 10.3 s, total: 7h 3min 24s
Wall time: 7h 4min 31s


In [11]:
cv.best_params_

{'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 50,
 'tfidf__use_idf': True,
 'vect__max_df': 1.0,
 'vect__ngram_range': (1, 1)}

In [12]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_pred = cv.predict(X_test)

def class_report(y_test, y_pred):
    for i, column in enumerate(y_test):
        print(column)
        print(classification_report(y_test[column], y_pred[:, i]))
        print('#####################################################\n')
        
class_report(y_test, y_pred)

related
             precision    recall  f1-score   support

          0       0.72      0.30      0.42      1266
          1       0.81      0.96      0.88      3978

avg / total       0.79      0.80      0.77      5244

#####################################################

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.93      4309
          1       0.87      0.42      0.56       935

avg / total       0.88      0.88      0.87      5244

#####################################################

offer
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      5178
          1       0.62      0.15      0.24        66

avg / total       0.98      0.99      0.98      5244

#####################################################

aid_related
             precision    recall  f1-score   support

          0       0.76      0.87      0.81      3073
          1       0.77      0.62      0.68      2

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
# data_dmatrix = xgb.DMatrix(data=X_train, label=y_train)

pipeline_xgb_cv = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(xgb.XGBClassifier(learning_rate=0.02, objective='binary:hinge')))
])

pipeline_xgb_cv.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fe3a5d44620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, gamma=None,
          gpu_id=None, importance_type='gain', interaction_constraints=None,
          learning_rate=0.02, max_delta_step=None, max_depth=None,
          min_child_w...pos_weight=None, subsample=None,
          tree_method=None, validate_parameters=False, v

In [16]:
%%time

parameters = {
    'clf__estimator__n_estimators': [20, 100],
    'clf__estimator__min_child_weight': [1, 5, 10],
    'clf__estimator__gamma': [0.5, 1, 2, 5],
    'clf__estimator__subsample': [0.6, 0.8, 1.0],
    'clf__estimator__colsample_bytree': [0.6, 0.8, 1.0],
    'clf__estimator__max_depth': [3, 4, 5]
}

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

xgb_randomcv = RandomizedSearchCV(pipeline_xgb_cv, param_distributions=parameters, n_iter=10,
                                  scoring='roc_auc', n_jobs=-1, cv=5, verbose=3, random_state=42)

xgb_randomcv.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] clf__estimator__subsample=1.0, clf__estimator__n_estimators=100, clf__estimator__min_child_weight=1, clf__estimator__max_depth=5, clf__estimator__gamma=5, clf__estimator__colsample_bytree=1.0 
[CV]  clf__estimator__subsample=1.0, clf__estimator__n_estimators=100, clf__estimator__min_child_weight=1, clf__estimator__max_depth=5, clf__estimator__gamma=5, clf__estimator__colsample_bytree=1.0, score=0.6556547571341669, total= 8.2min
[CV] clf__estimator__subsample=1.0, clf__estimator__n_estimators=100, clf__estimator__min_child_weight=1, clf__estimator__max_depth=5, clf__estimator__gamma=5, clf__estimator__colsample_bytree=1.0 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  8.4min remaining:    0.0s


[CV]  clf__estimator__subsample=1.0, clf__estimator__n_estimators=100, clf__estimator__min_child_weight=1, clf__estimator__max_depth=5, clf__estimator__gamma=5, clf__estimator__colsample_bytree=1.0, score=0.6439619811690237, total= 8.2min
[CV] clf__estimator__subsample=1.0, clf__estimator__n_estimators=100, clf__estimator__min_child_weight=1, clf__estimator__max_depth=5, clf__estimator__gamma=5, clf__estimator__colsample_bytree=1.0 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 16.9min remaining:    0.0s


[CV]  clf__estimator__subsample=1.0, clf__estimator__n_estimators=100, clf__estimator__min_child_weight=1, clf__estimator__max_depth=5, clf__estimator__gamma=5, clf__estimator__colsample_bytree=1.0, score=0.6540218517502543, total= 8.2min
[CV] clf__estimator__subsample=1.0, clf__estimator__n_estimators=100, clf__estimator__min_child_weight=1, clf__estimator__max_depth=5, clf__estimator__gamma=5, clf__estimator__colsample_bytree=1.0 
[CV]  clf__estimator__subsample=1.0, clf__estimator__n_estimators=100, clf__estimator__min_child_weight=1, clf__estimator__max_depth=5, clf__estimator__gamma=5, clf__estimator__colsample_bytree=1.0, score=0.6499461316239709, total= 8.2min
[CV] clf__estimator__subsample=1.0, clf__estimator__n_estimators=100, clf__estimator__min_child_weight=1, clf__estimator__max_depth=5, clf__estimator__gamma=5, clf__estimator__colsample_bytree=1.0 
[CV]  clf__estimator__subsample=1.0, clf__estimator__n_estimators=100, clf__estimator__min_child_weight=1, clf__estimator__max

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 215.1min finished


CPU times: user 3h 42min 55s, sys: 28.7 s, total: 3h 43min 23s
Wall time: 3h 44min


In [17]:
xgb_randomcv.best_params_

{'clf__estimator__subsample': 1.0,
 'clf__estimator__n_estimators': 100,
 'clf__estimator__min_child_weight': 10,
 'clf__estimator__max_depth': 5,
 'clf__estimator__gamma': 2,
 'clf__estimator__colsample_bytree': 1.0}

In [18]:
xgb_randomcv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ple=1.0,
       tree_method=None, validate_parameters=False, verbosity=None),
           n_jobs=1))])

In [20]:
y_pred = xgb_randomcv.predict(X_test)

def class_report(y_test, y_pred):
    for i, column in enumerate(y_test):
        print(column)
        print(classification_report(y_test[column], y_pred[:, i]))
        print('#####################################################\n')
        
class_report(y_test, y_pred)

related
             precision    recall  f1-score   support

          0       0.82      0.06      0.11      1266
          1       0.77      1.00      0.87      3978

avg / total       0.78      0.77      0.68      5244

#####################################################

request
             precision    recall  f1-score   support

          0       0.90      0.95      0.93      4309
          1       0.71      0.50      0.59       935

avg / total       0.86      0.87      0.87      5244

#####################################################

offer
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      5178
          1       0.20      0.05      0.07        66

avg / total       0.98      0.99      0.98      5244

#####################################################

aid_related
             precision    recall  f1-score   support

          0       0.74      0.81      0.77      3073
          1       0.69      0.59      0.64      2

### 9. Export your model as a pickle file

In [21]:
# save the model
filename = 'xgb_randomcv.pkl'

with open(filename, 'wb') as f:
    pickle.dump(xgb_randomcv, f)
    
# load the model
# with open(filename, 'rb') as f:
#     model = pickle.load(f)
    
# result = model.score(X_test, y_test)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.